In [1]:
!pip install webdriver-manager

In [2]:
!pip install selenium

In [3]:
!pip install html5lib

In [4]:
!pip install pandas

In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time 
import pandas as pd

In [1]:
# usando o chat gpt
import time

from selenium import webdriver

from selenium.webdriver.chrome.service import Service

service = Service('/path/to/chromedriver')

service.start()

driver = webdriver.Remote(service.service_url)


driver.get('http://www.facebook.com/');

time.sleep(60) # Let the user actually see something!

driver.quit()

In [6]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

#url = "https://www.b3.com.br/pt_br/"
url = "https://www.etf.com/etfanalytics/etf-finder"
driver.get(url)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 6.78M/6.78M [00:01<00:00, 4.60MB/s]


# Conhecer e mapear o processo de coleta de dados no site da EFT
https://www.etf.com/

Entraremos no site da etf 
Entraremos em databases
Colocar em vizualição por pagina em 100


# 5.1 Achar todos os elementos necessários detro do HTML - Expandindo a tabela para 100 por pagina


In [7]:
time.sleep(12)

botao100 = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao100)

# 5.2 Achar todos os elementos necessários dentro do HTML - Pegando o números de paginas na tabela

In [8]:
numeros_de_pag = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

# apos pegar o numeros de pagina deixamos apenas 
numeros_de_pag = numeros_de_pag.text.replace("of ","")
numeros_de_pag = int(numeros_de_pag)

print(numeros_de_pag)

32


# 6.1 Lendo a tabela de dados

In [9]:
# aprender a tabela 1

tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')

html = tabela.get_attribute("outerHTML")

# lendo html com pandas 
tabela_final = pd.read_html(html)[0]

display(tabela_final)

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$364.61B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$300.18B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$276.86B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$275.42B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$156.72B
...,...,...,...,...,...,...
95,XLI,Industrial Select Sector SPDR Fund,Equity: U.S. Industrials,State Street Global Advisors,0.10%,$13.90B
96,USFR,WisdomTree Floating Rate Treasury Fund,"Fixed Income: U.S. - Government, Treasury Inve...",WisdomTree,0.15%,$13.55B
97,SCHP,Schwab U.S. TIPS ETF,"Fixed Income: U.S. - Government, Inflation-lin...",Charles Schwab,0.04%,$13.54B
98,PFF,iShares Preferred and Income Securities ETF,"Fixed Income: U.S. - Corporate, Preferred",Blackrock,0.45%,$13.53B


In [13]:
# abrindo todas as paginas


# criando uma variavel para salvar a lista de paginas
# recebe uma lista vazia 
lista_de_paginas = []

# percorrendo todas as paginas
for pages in range(0, numeros_de_pag):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')

    html = tabela.get_attribute("outerHTML")

    # lendo html com pandas 
    tabela_final = pd.read_html(html)[0]
    
    # append para add um item na tabela
    lista_de_paginas.append(tabela_final)
    
    # botao avançar pagina
    botao_proxima_pagina = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]''')
    
    driver.execute_script("arguments[0].click();",botao_proxima_pagina)
    
base_de_dados_completa = pd.concat(lista_de_paginas)

display(base_de_dados_completa)

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$364.61B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$300.18B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$276.86B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$275.42B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$156.72B
...,...,...,...,...,...,...
3,CVSB,Calvert Ultra-Short Investment Grade ETF,"Fixed Income: Global - Broad Market, Broad-bas...",Morgan Stanley,0.24%,--
4,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--
5,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
6,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,0.18%,--
